In [98]:
family = 'RF00174' 
sequence = 'GUACUAUAUGCGACACUGUGUAUCUGUAAAGGCGAUGACGAUGCAGGAAGUCGGUGCAAAGCCGGCGCGGUCCCGCCACUGUAAUCGGGGAGCGACCCUCGUAGGCCACGGCCAUUGGCUGGAAGGCGAGGCAAGCGGUGAUCCGAGAGCCAGGAAACUCGCGUCAUCGCGUCCUGCUAAC'


In [99]:
kmer_norm_col = pd.read_csv('kmer_family.csv', index_col = 0)
kmer = kmer_norm_col.loc[family]

kmerlist = [0 for i in range(len(sequence))]
for feature in featurenames:
    p = 0
    step = len(feature) -1
    while p != (len(sequence) - step):
        if feature == sequence[p: p + step + 1]:
            for q in range(p, p+step+1):
                kmerlist[q] = kmerlist[q] + kmer[feature]
            p = p + 1
        else:
            p = p + 1

            
for i in range(0, len(sequence)):
    kmerlist[i] = (kmerlist[i] - min(kmerlist))/(max(kmerlist)-min(kmerlist))

In [100]:
print(kmerlist)
mkdir('./ribo_colormap/%s' % family)

with open(check_filename_available('./ribo_colormap/%s/%s.txt' % (family, family)), 'a', encoding="UTF-8") as ribo:
    for element in kmerlist:
        ribo.write('%.2f' % element + '\n')

[0.1519098005193257, 0.1519098005193257, 0.168767154529692, 0.2596139389848647, 0.13399073738908054, 0.25180775276170386, 0.20658447071104297, 0.16096096830653117, 0.2742590149336931, 0.3263379623674996, 0.1698959628064298, 0.2351464916656953, 0.4039136461953873, 0.5814358879063756, 0.7450499051679802, 0.6721730974041563, 0.6598479424355341, 0.8117577429548598, 0.4719658173420002, 0.30835180008039553, 0.1519098005193257, 0.23290755944960104, 0.23290755944960104, 0.28498650688340754, 0.44142850644447734, 0.6481437256932551, 0.3567430293118041, 0.41438519275004954, 0.5504963140859827, 0.46976514347464826, 0.33365402213871503, 0.3311074644781951, 0.2576666218565572, 0.1622696839018713, 0.2800866992744946, 0.2742590149336931, 0.3187116834629411, 0.4026429286342732, 0.24037324473240185, 0.23387577410458116, 0.35169278947720445, 0.32884893826777994, 0.3809278857015864, 0.3943818489469464, 0.3619911406214177, 0.19209517781498794, 0.1622696839018713, 0.2983808052378044, 0.37887904744168643, 0.

In [101]:
b = {'CCCGC', 'AGUCAG', 'ACCUGC', 'GGAAGG', 'GGGAAG', 'CACUG', 'GCCACU'}

In [102]:
for feature in b:
    index = 0
    while index < len(sequence)-1:
        if sequence.find(feature, index, len(sequence)-1) != -1:
            print(feature)
            print(sequence.find(feature, index, len(sequence)-1))
            index = sequence.find(feature, index, len(sequence)-1)+1
        else:
            index = len(sequence)

CACUG
13
CACUG
76
GGAAGG
120
GCCACU
74
CCCGC
71


In [84]:
sequence

'GUCGGUAGCAUUCUGUGCGUUGGUCCAGGGAACCCGGUGAGAGUCCGGGGCUGUCCCGCAACGGUGUGACCUUUUCGCCCCUCGGCGCCGCGGCGCCCCCGCGGGCGAGGUCGAGCCCGAUACCUGCCGAC'

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib, os


def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)

    if not isExists:
        os.makedirs(path)
        return True
    else:
        return False

    
def check_filename_available(filename):
    n=[0]
    def check_meta(file_name):
        file_name_new=file_name
        if os.path.isfile(file_name):
            file_name_new=file_name[:file_name.rfind('.')]+'_'+str(n[0])+file_name[file_name.rfind('.'):]
            n[0]+=1
        if os.path.isfile(file_name_new):
            file_name_new=check_meta(file_name)
        return file_name_new
    return_name=check_meta(filename)
    return return_name


def preprocess(countpath, testsize = 0.2, randomstate = 1):
    x = pd.read_csv('%s' % countpath, index_col=0).T

    # get feature names
    featurenames = np.array(x.index)
    
    # get label names
    x = x.T
    y = list(np.array(x.index))
    label = sorted(list(set(y)), key = y.index)
    
    df = x
    df['target'] = y

    for classname in label:
        count_temp = len(df[df.target == classname])
        print("Proportion of family {0}: {1:.2f}%".format(classname, (count_temp / (len(df.target)) * 100)))


    y = df.target.values
    x= df.drop(['target'], axis=1)
    
    
    x = x.dropna(axis=1, how='any')  # drop all rows that have any NaN value
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = testsize, random_state = randomstate)

    return x_train, y_train, x_test, y_test, featurenames, label, x


def counting(label, y_train):
    counting = {}
    for i in label:
        count = 0
        for j in y_train:
            if i == j:
                count = count+1        
        counting[i] = count
    print(counting)
    
    return counting

def heat(label, featurenames, x_train, counting):
    heat = pd.DataFrame(0, index=np.arange(len(label)), columns=np.arange(featurenames.size))
    heat.index = label
    heat.columns = featurenames
    for i in range(0, len(x_train.index)):
        for j in range(0, len(x_train.columns)):
            heat.loc[x_train.index[i], x_train.columns[j]] = heat.loc[x_train.index[i], x_train.columns[j]] + x_train.iloc[i, j]

    return heat

In [26]:
if __name__ == '__main__':
    countpath = './156_new_selected_156.csv'

    x_train, y_train, x_test, y_test, featurenames, label, dataframe = preprocess(countpath, 0.3, 3)
    
    heat = heat(label, featurenames, x_train, y_train)

count = counting(label, y_train)

heat_norm = heat
for i in range(0, len(heat_norm.index)):
    for j in range(0, len(heat_norm.columns)):
        heat_norm.iloc[i, j] = heat_norm.iloc[i, j] / count[heat_norm.index[i]]

heat_norm_col = heat_norm
heat_norm_col=(heat_norm_col-heat_norm_col.min())/(heat_norm_col.max()-heat_norm_col.min())

heat_norm_col.to_csv('kmer_family.csv')